# Mineral Deposits and Sediment Samples - Exploratory Data Analysis
This notebook performs exploratory data analysis including:<br>
1. Descriptive statistics for each element (mean, median, standard deviation, count) and distribution.<br>
2. Box plots broken up by element category (major, minor, trace).<br>
3. Correlation Matrix.<br>

In [14]:
import pandas as pd
import numpy as np

### Load in Data
For testing purposes we are just loading in the first 10000 entries of the AKSediment csv, called 'AKSediment_truncated.csv'.

In [15]:
sediment_csv = 'data/AKSediment_truncated.csv'
mindeposit_csv = 'data/original/AKMinDeposits.csv'

sediment_df = pd.read_csv(sediment_csv)
mindeposit_df = pd.read_csv(mindeposit_csv)

# Print out how many rows/columns are in the dataframes and also the first few rows of the dataframes,uncomment as needed
# print(sediment_df.head(10))
print("Number of rows in sediment_df:", sediment_df.shape[0])
print("Number of columns in sediment_df:", sediment_df.shape[1])
# print(mindeposit_df.head(10))
print("Number of rows in mindeposit_df:", mindeposit_df.shape[0])
print("Number of columns in mindeposit_df:", mindeposit_df.shape[1])

C:\Users\Eleonora\AppData\Local\Temp\ipykernel_5780\2502602361.py:4: DtypeWarning: Columns (9,18,19,35,36,39,45,71,72,104,105,128,129,137,138,143,144,149,150,161,162,164,165,167,168,173,174,191,192,197,198,206,207,212,213,215,216,218,219,224,225,227,228,230,231,245,246,248,249,257,258,263,264,266,267,272,273,278,279,290,291) have mixed types. Specify dtype option on import or set low_memory=False.
  sediment_df = pd.read_csv(sediment_csv)


Number of rows in sediment_df: 10000
Number of columns in sediment_df: 300
Number of rows in mindeposit_df: 7720
Number of columns in mindeposit_df: 30


### Data Cleaning and Validation for Mineral Deposit Data
First we will filter by location and then we will filter by mineral. <br><br>

We will start by looking just at the mineral deposits in the Brooks Range area. The lat and long boundaries used are rough testing boundaries.

In [16]:
# Define latitude and longitude boundaries
min_lat, max_lat = 65, 69
min_lon, max_lon = -166, -141

# Filter the DataFrame for rows within the specified range
mindeposit_area_filtered_df = mindeposit_df[(mindeposit_df['latitude'] >= min_lat) & 
                        (mindeposit_df['latitude'] <= max_lat) & 
                        (mindeposit_df['longitude_for_GIS'] >= min_lon) & 
                        (mindeposit_df['longitude_for_GIS'] <= max_lon)]

# # Output the result for testing, uncomment as needed
# mindeposit_area_filtered_df.to_csv('mindeposit_area_filtered.csv', index=False)
# print("Number of rows in mindeposit_area_filtered_df:", mindeposit_area_filtered_df.shape[0])
# print("Number of columns in mindeposit_area_filtered_df:", mindeposit_area_filtered_df.shape[1])

We will start by looking just at copper and zinc deposits in the mineral deposits data.

In [17]:
mindeposit_mineral_filtered_df = mindeposit_area_filtered_df[
    (mindeposit_area_filtered_df['commodities_main'].str.contains('Cu', na=False) | 
     mindeposit_area_filtered_df['commodities_other'].str.contains('Cu', na=False)) |
    (mindeposit_area_filtered_df['commodities_main'].str.contains('Zn', na=False) | 
     mindeposit_area_filtered_df['commodities_other'].str.contains('Zn', na=False))
]

# # Output the result for testing, uncomment as needed
# mindeposit_mineral_filtered_df.to_csv('mindeposit_mineral_filtered.csv', index=False)
# print("Number of rows in mindeposit_mineral_filtered_df:", mindeposit_mineral_filtered_df.shape[0])
# print("Number of columns in mindeposit_mineral_filtered_df:", mindeposit_mineral_filtered_df.shape[1])

# Save the cleaned data to the /cleaned dir
mindeposit_mineral_filtered_df.to_csv('AKMinDeposits_cleaned.csv', index=False)

### Data Cleaning and Validation for Sediment Data
We will start by looking just at the mineral deposits in the Brooks Range area. The lat and long boundaries used are rough testing boundaries.

In [18]:
# Define latitude and longitude boundaries
min_lat, max_lat = 65, 69
min_lon, max_lon = -166, -141

# Filter the DataFrame for rows within the specified range
sediment_area_filtered_df = sediment_df[(sediment_df['LATITUDE'] >= min_lat) & 
                        (sediment_df['LATITUDE'] <= max_lat) & 
                        (sediment_df['LONGITUDE'] >= min_lon) & 
                        (sediment_df['LONGITUDE'] <= max_lon)]

# # Output the result for testing, uncomment as needed
# sediment_area_filtered_df.to_csv('sediment_area_filtered.csv', index=False)
# print("Number of rows in sediment_area_filtered_df:", sediment_area_filtered_df.shape[0])
# print("Number of columns in sediment_area_filtered_df:", sediment_area_filtered_df.shape[1])

We will grab only the rows we are interested in: LATITUDE, LONGITUDE, and chemical elements in ppm or pct.

In [19]:
# Columns to explicitly keep
columns_to_keep = ['LATITUDE', 'LONGITUDE']

# Filter for columns that end with '_ppm' or '_pct'
filtered_columns = sediment_area_filtered_df.filter(regex='(_ppm|_pct)$').columns

sediment_column_filtered_df = sediment_area_filtered_df[columns_to_keep + list(filtered_columns)] 

# # Output the result for testing, uncomment as needed
# sediment_column_filtered_df.to_csv('sediment_column_filtered.csv', index=False)
# print("Number of rows in sediment_column_filtered_df:", sediment_column_filtered_df.shape[0])
# print("Number of columns in sediment_column_filtered_df:", sediment_column_filtered_df.shape[1])

Now we will convert the values stored as percentages to ppm. Before doing that, we verify that all columns containing ppm or pct values are of type float.

In [20]:
# pct to ppm scaling factor
PCT_T0_PPM = 10000 # TODO CHECK THIS WITH JORDAN

# Filter columns ending with '_ppm' or '_pct'
filtered_columns = sediment_column_filtered_df.filter(regex='(_ppm|_pct)$').columns

# Check if all these columns are of type float
ALL_FLOATS = sediment_column_filtered_df[filtered_columns].map(lambda x: isinstance(x, float)).all().all()

# If all these columns are of type float, convert pct columns to ppm
if ALL_FLOATS:
    print('All columns ending in _ppm or _pct are of type float.')
    
    # Find all columns ending with '_pct'
    pct_columns = sediment_column_filtered_df.filter(regex='_pct$').columns

    sediment_pct_to_ppm_df = sediment_column_filtered_df.copy()

    # Multiply values in the _pct columns by the scaling factor
    sediment_pct_to_ppm_df.loc[:, pct_columns] *= PCT_T0_PPM

    # Rename the columns ending in _pct to ending in _ppm
    sediment_pct_to_ppm_df.rename(columns=lambda x: x.replace('_pct', '_ppm') if x.endswith('_pct') else x, inplace=True)

    # # Output the result for testing, uncomment as needed
    # sediment_pct_to_ppm_df.to_csv('sediment_pct_to_ppm.csv', index=False)
    # print("Number of rows in sediment_pct_to_ppm_df:", sediment_pct_to_ppm_df.shape[0])
    # print("Number of columns in sediment_pct_to_ppm_df:", sediment_pct_to_ppm_df.shape[1])

All columns ending in _ppm or _pct are of type float.


Values below -1000 should be replaced with NaN as they are likely errors.

In [21]:
# Filter columns ending with '_ppm' or '_pct'
filtered_columns = sediment_pct_to_ppm_df.filter(regex='_ppm$').columns

sediment_adjusted_values_nan_df = sediment_pct_to_ppm_df.copy()

# Replace values below and including -1000 with NaN
sediment_adjusted_values_nan_df[filtered_columns] = sediment_adjusted_values_nan_df[filtered_columns].mask(sediment_adjusted_values_nan_df[filtered_columns] <= -1000, np.nan)

# Unit test using assert
for column in filtered_columns:
    # Check if there are any values below or equal to -1000
    below_threshold = (sediment_adjusted_values_nan_df[column] <= -1000)
    # Count of values below or equal to -1000
    count_below_threshold = below_threshold.sum()
    # Assert that count is 0
    assert count_below_threshold == 0, f"Column '{column}' contains {count_below_threshold} values below or equal to -1000."

print("All values below and including -1000 have been successfully removed.")

# # Output the result for testing, uncomment as needed
# sediment_adjusted_values_nan_df.to_csv('sediment_adjusted_values_nan.csv', index=False)
# print("Number of rows in sediment_adjusted_values_nan_df:", sediment_adjusted_values_nan_df.shape[0])
# print("Number of columns in sediment_adjusted_values_nan_df:", sediment_adjusted_values_nan_df.shape[1])

All values below and including -1000 have been successfully removed.


Other negative values should be replaced with half their magnitude (half the detection limit). 

In [22]:
sediment_adjusted_values_detection_df = sediment_adjusted_values_nan_df.copy()

# For other negative values, replace them with half their maginute (approximately half of the detection limit)
sediment_adjusted_values_detection_df[filtered_columns] = sediment_adjusted_values_detection_df[filtered_columns].mask(
    sediment_adjusted_values_detection_df[filtered_columns] < 0, abs(sediment_adjusted_values_detection_df[filtered_columns] / 2))

# # Output the result for testing, uncomment as needed
# sediment_adjusted_values_detection_df.to_csv('sediment_adjusted_values_detection.csv', index=False)
# print("Number of rows in sediment_adjusted_values_detection_df:", sediment_adjusted_values_detection_df.shape[0])
# print("Number of columns in sediment_adjusted_values_detection_df:", sediment_adjusted_values_detection_df.shape[1])

# Save the cleaned data to the /cleaned dir
sediment_adjusted_values_detection_df.to_csv('AKSediment_cleaned.csv', index=False)